# Primera entrega ML

### Título de proyecto

In [1]:
# Escribir el título de proyecto en Markdown
titulo = "Estimador del valor de las compañias del SP500 en función a sus estados financieros y la coyuntura"

### Tema

In [ ]:
tema = "Estimar el valor de una compañia del SP500 en bolsa en base aa sus estados financieros" # Breve descripción del proyecto

#### Describir el tema en Markdown 
Debido a la magnituid del proyecto el objetivo es construir y entrenar un estimador lo antes posible. Los pasos son
1. Eliminar observaciones con errores de medida
2. Clip a los percentiles 1 y 99 de outliers
3. Eliminamos variables que estén correlacionadas
4. MinMax scaling para que ningun dato sea negativo y poder hacer una transformación logaritmica
5. LogTransform porque todas las variables tienen una asimetría muy grande.
6. Entrenar un modelo de regresion Para obtener resultados, entrenar también un RandomForest
7. A partir de los dos modelos anteriores obtener la importancia de los features

Con los resultados de la importnacia de los features ya podemos:
1. Eliminar features redundantes
2. Centrarnos en limpiar bien los más importantes
3. Hacer feature engineering. 
   1. Obtener variables que reflejen el % incremento con respecto a otros años
   2. Obtener variables de la coyuntura económica
   3. Obtener ratios financieros
   4. Si son muchos features, PCA para eliminar posibles multicolinealidades
   5. etc
4. Para ese dataset probar con optuna a hacer trials de 30 mins de:
   1. ElasticNet
   2. Random Forest
   3. KNN
   4. SVC. Lineal Polynomial y rbf
   5. Adaboost
   6. xgboost
5. Si no da buenos resultados intentar combinar 3 o 4 en un modelo con <a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html">el objeto de stacking de sklearn</a>

Iterar

### Datasets y fuentes alternativas de datos
Incluye aquí una breve descripción del dataset o datasets para tu proyecto. Incluye también las fuentes de cada uno de ellos.

In [ ]:
tipo_ml = "regresion" # Regresión? Clasificación?
target_descr = "Simple moving average delm valor de la compañia 10 días despues de publicar resultados" # Descripción de la variable objetivo
metricas = ['r2', 'mean absolute error', 'mean squared error'] # Metricas escogidas para evaluar los modelos

In [ ]:
# Los datasets ocupan demasiado. Enlazo link a mi OneDrive
# los datos se han obtenido de 
# https://www.sec.gov/edgar/searchedgar/companysearch.html, 
# https://site.financialmodelingprep.com/developer/docs, 
# https://en.wikipedia.org/wiki/List_of_S%26P_500_companies, 
# https://fred.stlouisfed.org/categories
enlaces = ['https://1drv.ms/u/s!Ag5scVmolBSPh5hFEVEKfXakuVYBsQ?e=X7SEJb']

EL principal dataset sobre el que se va a entrenar el modelo es

In [2]:
import pandas as pd

In [17]:
data = pd.read_csv('src\data\EDA\statements_mcap.csv')

In [18]:
data.shape

(9490, 112)

In [23]:
data.isna().values.sum()

0

In [25]:
data.sample(10)

,symbol,calendarYear,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,...,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,month,inflationMultiplier,mcapDate,mcapYear,mcapWeek,target
2191,CSCO,2007,4.862152,24.177727,29.039879,5.202554,1.724186,5.212988,41.179607,5.077349,...,4.300031,13.177892,-1.631586,11.546306,7.0,1.304225,2007-09-28,2007.0,39.0,267.329302
3363,FDX,2008,1.971171,0.000000,1.971171,5.583063,0.557154,1.166820,9.278207,17.262793,...,2.009595,4.462351,-3.774555,0.687796,5.0,1.280813,2008-07-26,2008.0,30.0,30.816244
1132,BC,2002,0.504034,0.000000,0.504034,0.584215,0.784452,0.508624,2.381326,1.137018,...,0.155628,0.592391,-0.161509,0.430882,12.0,1.434361,2003-01-10,2003.0,2.0,2.646495
5321,LOW,2011,0.809097,0.584486,1.393583,0.000000,10.325916,0.649015,12.368514,27.411268,...,0.784278,4.780126,-1.649218,3.130908,1.0,1.240947,2011-04-08,2011.0,14.0,44.040537
5591,MCHP,2019,0.465079,0.002496,0.467575,0.955549,0.772274,0.207907,2.403305,1.081531,...,0.978011,1.817345,-0.248382,1.568963,3.0,1.085112,2019-06-09,2019.0,23.0,22.345224
2148,CPB,2020,0.909890,0.000000,0.909890,0.609065,0.922601,0.084739,2.526295,2.777336,...,0.032837,1.478703,-0.316714,1.161990,8.0,1.059243,2020-10-04,2020.0,40.0,15.030843
1825,CLF,2013,0.395792,0.000000,0.395792,0.318521,0.716554,0.409476,1.840344,13.157749,...,0.230279,1.351827,-1.016436,0.335391,12.0,1.179707,2014-02-24,2014.0,9.0,4.235823
5533,MAT,2011,1.669665,0.000000,1.669665,1.520363,0.593908,0.415744,4.199679,0.638958,...,1.562359,0.810608,-0.232824,0.577784,12.0,1.219523,2012-03-04,2012.0,9.0,13.557015
6918,PGR,2000,0.013217,0.277418,0.290636,2.680176,0.000000,0.000000,0.000000,0.749237,...,0.021089,1.221353,-0.193510,1.027844,12.0,1.485109,2001-01-10,2001.0,2.0,10.978893
368,AFL,2006,1.589469,0.000000,1.589469,0.706871,0.000000,0.000000,0.000000,0.605134,...,1.713667,5.809555,-0.030389,5.779166,12.0,1.321254,2007-03-10,2007.0,10.0,31.069014


In [26]:
data.columns.tolist()

['symbol',
 'calendarYear',
 'cashAndCashEquivalents',
 'shortTermInvestments',
 'cashAndShortTermInvestments',
 'netReceivables',
 'inventory',
 'otherCurrentAssets',
 'totalCurrentAssets',
 'propertyPlantEquipmentNet',
 'goodwill',
 'intangibleAssets',
 'goodwillAndIntangibleAssets',
 'longTermInvestments',
 'taxAssets',
 'otherNonCurrentAssets',
 'totalNonCurrentAssets',
 'otherAssets',
 'totalAssets',
 'accountPayables',
 'shortTermDebt',
 'taxPayables',
 'deferredRevenue',
 'otherCurrentLiabilities',
 'totalCurrentLiabilities',
 'longTermDebt',
 'deferredRevenueNonCurrent',
 'deferredTaxLiabilitiesNonCurrent',
 'otherNonCurrentLiabilities',
 'totalNonCurrentLiabilities',
 'otherLiabilities',
 'capitalLeaseObligations',
 'totalLiabilities',
 'preferredStock',
 'commonStock',
 'retainedEarnings',
 'accumulatedOtherComprehensiveIncomeLoss',
 'othertotalStockholdersEquity',
 'totalStockholdersEquity',
 'totalLiabilitiesAndStockholdersEquity',
 'minorityInterest',
 'totalEquity',
 'tot